This notebook demonstrates audio output from Julia using the AudioIO.jl package.

In [ ]:
using AudioIO

Let's start by defining the concept of a musical note.

In [33]:
using SIUnits
typealias timeperiod{T} quantity(T,Second)
typealias frequency {T} quantity(T,Hertz)

immutable note{T<:Real}
    pitch::frequency{T}     #has dimensions of inverse time
    duration::timeperiod{T} #has dimensions of time
    sustained::Bool         #Do we want the note to last the entire time period?
end

#Outer constructor
function note{S<:Real,T<:Real}(pitch::frequency{S}, duration::timeperiod{T},
        sustained::Bool=false)

    R=promote_type(S,T)
    note(convert(frequency{R}, pitch), convert(timeperiod{R}, duration), sustained)
end

LoadError: invalid redefinition of constant note
while loading In[33], in expression starting on line 5

Now we have the concept of a musical note. Let's define how play it.

We generate (using `waveform()`) a sample from a sine wave of the given frequency and that has the appropriate duration. We then send the generated waveform to AudioIO to be played.

For convenience, we define a new method `play(::Note)` that allows us to play a musical note.

In [3]:
import AudioIO.play
import Base.sleep
sleep{T<:Real}(time::timeperiod{T})=sleep(time/Second)

function play{T<:Real}(A::note{T}; 
        samplingfreq::frequency{T}=convert(T,44100)*Hertz, shape::Function=sin)

    play(waveform(A, samplingfreq=samplingfreq, shape=shape))
    sleep(A.duration)
end

function waveform{T<:Real}(A::note{T}; 
        samplingfreq::frequency{T}=convert(T,44100)Hertz, shape::Function=sin)

    timesamples=0Second:1/samplingfreq:(A.duration*(A.sustained ? 1.0 : 0.93))
    wf = zeros(Float32, length(0Second:1/samplingfreq:A.duration))
    for (i, t) in enumerate(timesamples)
        wf[i] = shape(2π*A.pitch*t)
    end
    wf
end

waveform (generic function with 1 method)

In [6]:
methods(play)

# 9 methods for generic function "play":
play(arr::Array{Float32,N},args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/nodes.jl:226
play{T<:FloatingPoint}(arr::Array{T<:FloatingPoint,N},args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/nodes.jl:232
play{T<:Signed}(arr::Array{T<:Signed,N},args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/nodes.jl:241
play{T<:Unsigned}(arr::Array{T<:Unsigned,N},args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/nodes.jl:246
play(filename::String,args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/sndfile.jl:208
play(file::AudioFile,args...) at /Users/jiahao/.julia/v0.3/AudioIO/src/sndfile.jl:213
play(node::AudioNode{T<:AudioRenderer},stream::AudioStream) at /Users/jiahao/.julia/v0.3/AudioIO/src/AudioIO.jl:70
play(node::AudioNode{T<:AudioRenderer}) at /Users/jiahao/.julia/v0.3/AudioIO/src/AudioIO.jl:76
play{T<:Real}(A::note{T<:Real}) at In[3]:7

[Modern musical convention](https://en.wikipedia.org/wiki/A440_(pitch_standard)) defines the musical note A to have pitch 440 Hertz.

Here is concert A defined for a duration of 1 second.

In [34]:
#Concert A
A = note(440Hertz, 1Second)

note{Int64}(440 s⁻¹,1 s ,false)

And now we can hear what it sounds like.

In [35]:
play(A)

And we can change its timbre by changing the waveform used to generate it. (We used a pure sine wave by default.)

In [36]:
play(note(440Hertz, 1Second),
    shape=t->0.4sin(t)+0.3sin(3t)+0.2cos(5t)+0.12sin(6t)+0.003sin(12t))

To play other notes, we need the notion of a musical scale. Most Western music uses the [twelve-tone even-tempered scale](https://en.wikipedia.org/wiki/Equal_temperament). Two notes, where one note has twice the frequency of the other, span an octave. Divide the octave logarithmically into 12 even parts to get other notes.

In [7]:
#The even-tempered scale
B♭=note(440Hertz*2^(1//12), 1Second)
play(B♭)
B=note(440Hertz*2^(2//12), 1Second)
play(B)
C=note(440Hertz*2^(3//12), 1Second)
play(C)

If we leave out some of the notes, choosing only 8, we get a scale that fits an octave.

In [8]:
#C Major scale
for s in [-9, -7, -5, -4, -2, 0, 2, 3] #do re mi fa so la ti do
    play(note(440Hertz*2^(s//12), 0.5Second))
end

Putting together pitch and duration gives us the ability to play simple tunes.

In [9]:
#Doe, a deer, a female deer
for (s, t) in [(1,3), (2,1), (3,3), (1,1), (3,2), (1,2), (3,4)]
    play(note(440Hertz*2^((2s-11)//12), t*0.2Second))
end

# Playing music

Songs often come with lyrics. We can put the notes together with lyrics to define a voice line, and then define how to play the entire voice line.

In [10]:
immutable Voice{S<:Real}
    wf :: Vector{S} #Waveform
    lyrics :: Vector
end

In [23]:
function play(v::Voice, samplingfreq=44100Hertz)
    totalduration=length(v.wf)/samplingfreq
    play(v.wf)
    lyrics_start=time_ns()
    for (syllable, duration) in v.lyrics
        syllable_start=time_ns()
        
        #Print syllable, if there is something to print
        #Special case syllables ending in hyphens - they
        #represent continuations onto the next syllable.
        #Don't print them.
        if length(syllable)≥1
            if syllable[end]=='-'
                print(strip(syllable[1:end-1]))
            else
                print(strip(syllable), " ")
            end
        end
        
        #Pause for the length of syllable
        sleep(max(0Second, duration-(time_ns()-syllable_start)*1e-9Second))
    end
    
    #Wait for entire waveform to finish playing, if it hasn't already
    sleep(max(0Second, totalduration-time_ns()*1e-9Second))
end

play (generic function with 11 methods)

We'll also teach Julia how to read music. For convenience, here's a parser for a small subset of [the notation used by Lilypond](http://www.lilypond.org).

In [12]:
#Let's play some music written in Lilypond notation
function parsevoice(melody::String; tempo=132, beatunit=4, lyrics=nothing, octave=3)
    lyrics_syllables = lyrics==nothing? nothing : split(lyrics)

    note_idx = 1
    oldduration = 4
    Lyrics = Any[]
    Notes = Any[]
    for line in split(melody, '\n')
        percent_idx = findfirst(line, '%') #Trim comment
        percent_idx == 0 || (line = line[1:percent_idx-1])
        for token in split(line)
            pitch, duration, sustained, dotted=parsetoken(token, 440*2.0^(octave-4))
            duration==nothing && (duration = oldduration)
            oldduration = duration
            for i=1:dotted
                duration /= (1. + .5^i)
            end
            timeduration = (beatunit/duration)*(60/tempo)*Second
            if lyrics_syllables!=nothing && 1<=note_idx<=length(lyrics_syllables) #Print the lyrics, omitting hyphens
                push!(Lyrics, (lyrics_syllables[note_idx], timeduration))
            else
                push!(Lyrics, ("", timeduration))
            end
            push!(Notes, note(pitch*Hertz, timeduration, sustained))
            note_idx += 1
        end
        lyrics_syllables!=nothing && 1<=note_idx<=length(lyrics_syllables) && push!(Lyrics, ("\n", 0Second))
    end
    Voice([map(waveform, Notes)...], Lyrics)
end

function parsetoken(token::String, Atuning::Real=220)
    state = :findpitch
    pitch = 0.0
    sustain = false
    dotted = 0
    lengthbuf = Char[]
    for char in token
        if state == :findpitch
            scale_idx = findfirst('a':'g', char) + findfirst('A':'G', char)
            if scale_idx!=0
                const halfsteps = [12, 14, 3, 5, 7, 8, 10]
                pitch = Atuning*2^(halfsteps[scale_idx]/12)
                state = :findlength
            elseif char=='r'
                pitch, state = 0, :findlength
            else
                error("unknown pitch: $char")
            end
        elseif state == :findlength
            if     char == '#' ; pitch *= 2^(1/12) #sharp
            elseif char == 'b' ; pitch /= 2^(1/12) #flat
            elseif char == '\''; pitch *= 2        #higher octave
            elseif char == ',' ; pitch /= 2        #lower octave
            elseif char == '.' ; dotted += 1       #dotted note
            elseif char == '~' ; sustain = true    #tied note
            else
                push!(lengthbuf, char)
                #Check for "is" and "es" suffixes for sharps and flats
                if length(lengthbuf) >= 2
                    if lengthbuf[end-1:end] == "is"
                        pitch *= 2^(1/12)
                        lengthbuf = lengthbuf[1:end-2]
                    elseif lengthbuf[end-1:end] == "es"
                        pitch /= 2^(1/12)
                        lengthbuf = lengthbuf[1:end-2]
                    end
                end
            end
        end
    end
    #finalize length
    lengthstr = convert(String, lengthbuf)
    duration = isempty(lengthstr) ? nothing : parse(Int, lengthstr)
    return (pitch, duration, sustain, dotted)
end

parsetoken (generic function with 2 methods)

Here's what "Doe, a deer" looks like in Lilypond notation.

In [37]:
play(parsevoice("c. d8 e4. c8 e4 c e2", lyrics="Doe, a deer, a fe- male deer"))

Doe, a deer, a female deer 

Now let's play a little Beethoven.

In [25]:
play(parsevoice("f# f# g a a g f# e d d e f# f#~ f#8 e e2"))

In [26]:
solo=parsevoice("""
f# f# g a a g f# e d d e f# f#~ f#8 e e2
f#4 f# g a a g f# e d d e f# e~ e8 d d2
e4 e f# d e f#8~ g8 f#4 d e f#8~ g f#4 e d e a,
f#2 f#4 g a a g f# e d d e f# e~ e8 d8 d2""",
lyrics="""
Freu- de, schö- ner Göt- ter- fun- ken, Toch- ter aus E- li- - si- um!
Wir be- tre- ten feu- er- trun- ken, Himm- li- sche, dein Hei- - lig- thum!
Dei- ne Zau- ber bin den - wie- der, was die - Mo- de streng ge- theilt,
al- le mensch- en wer- den Brü- der wo dein sanf- ter Flü- - gel weilt.
""")
play(solo)

Freude, schöner Götterfunken, Tochter aus Elisium!  Wir betreten feuertrunken, Himmlische, dein Heiligthum!  Deine Zauber bin den wieder, was die Mode streng getheilt,  alle menschen werden Brüder wo dein sanfter Flügel weilt. 

And now in four-part harmony.

In [27]:
@sync begin
    @async soprano = play(parsevoice("""
f'#. f'#. g'. a'. a'. g'. f'#. e'~ e'8 d.'4 d.' e.' f#'. f#'.~ f#' e'8 e'4~ e'2
""", lyrics="Freu- de, schö- ner Göt- ter- fun- ken, Toch- ter aus E- li- - si- um!"
))

    @async alto = play(parsevoice("""
a. a. a. a.  a.  a. a. a~ g8 f#.4 a.  a.  a. a.~ a a8 a4~ a2
"""))

    @async tenor = play(parsevoice("""
d. d. e. f#. f#. e. d. d~ e8 f#.4 f#. a,. d. d.~ d c#8 c#4 c#2
"""))
    @async bass = play(parsevoice("""
d. d. d. d. a,. a,. a,. b,~ c8 d. a., a., a., a., a, a8, a,4 a,2
"""))

end

Freude, schöner Götterfunken, Tochter aus Elisium! 

Task (done) @0x00007f9fc3d96f20

In [28]:
@sync begin 
    soprano = @async play(parsevoice("""
    f'#.4 f'#. g'. a'. a'. g'. f'#. e'. d'. d'. e'. f'#. e'.~ e' d'8 d'4~ d'2
    """, lyrics="Wir be- tre- ten feu- er- trun- ken, Himm- li- sche, dein Hei- - lig- thum!"))

    alto = @async play(parsevoice("""
    a.4 a. b. c'. c'. b. a. g. f#. f#. g. f#. g.~ g4 f#8 f#~ f#2 
    """))

    tenor = @async play(parsevoice("""
    d.4 d. d. d. d. d. d. d. d. d. c#. d. c#.~ c# d8 d d2  
    """))
    bass = @async play(parsevoice("""
    d.4 d. d. d. a,. a,. a,. a., a., a., a., a., a.,~ a, a,8 d, d,2
    """))
end

Wir betreten feuertrunken, Himmlische, dein Heiligthum! 

Task (done) @0x00007f9fc4930aa0

# Opening of _Dies Irae_
## Verdi, Requiem

In [29]:
@sync begin
soprano1 = @async play(parsevoice("""
r2 r1
g1~ g~ g~ g~
""", octave=5))
soprano2 = @async play(parsevoice("""
r2 r1
g2~ f#4 f e~ eb~ d~ c~ d6~ c~ bb,~ c d eb d c bb, c4.. d16 d8
""", octave=5))
alto1 = @async play(parsevoice("""
r2 r1
g1~ g~ g~ g~
"""))
alto2 = @async play(parsevoice("""
r2 r1
g2~ f#4 f e eb d c d6 c bb, c d eb d c bb, c4.. d16 d8
"""))
tenor1 = @async play(parsevoice("""
eb.. eb16 e4.. e16 f4 f#
g1~ g~ g~ g~
"""))
tenor2 = @async play(parsevoice("""
eb.. eb16 e4.. e16 f4 f#
g2~ f#4 f e~ eb~ d~ c d6~ c bb, c d eb d c bb, c4.. d16 d8
    """
))
bass = @async play(parsevoice("""
eb.. eb16 e4.. e16 f4 f#  
g1~ g~ g~ g~
""", octave=3))
end

Task (done) @0x00007f9fc45d0660

# _Jesu, meine Freude_
## J. S. Bach, BWV 227, Nr. 1

In [30]:
@sync begin

soprano = @async play(parsevoice("""
b b a g f#2 e2.
b4 c#' d' b e'2 d#'2.
e'8 f#' g'4 f#.' f#8' e1.'
b4 b a g f#2 e.
b4 c#' d' b e'2 d#'2.
e'8 f#' g'4 f#.' f#8' e1.'
b4 b c' b a a8 g2.
b4 c#' d' b e' d'8~ c#' c#'2 b2.
b4 b a g8~ f# f#2 e1.
""", octave=3))

alto = @async play(parsevoice("""
g f# e8~ d# e4 e~ d# b2,.
g8~ f# e4 d d g8~ a~ b4 b2.
g8~ a b4 b. a8 g1.
g4 f# e8~ d# e4 e~ d# b2,.
g8~ f# e4 d d g8~ a~ b4 b2.
g8~ a~ b4 b. a8 g1.
g4 g a g g f# d2.
g4 g a g8~ a b4 b b~ a#~ f#2.
g4 f# e e e~ d# b,1.
""", octave=3))

tenor = @async play(parsevoice("""
e' b c'8~ f# g4 c'~ b8~ a g2.
e'8~ d' c#'~ b a4 g8~ a b4~ g' f#2'.
e'4 e' e' d#' b1.
e'4 b c'8~ f# g4 c'4~ b8~ a g2.
e'8~ d' c#'~ b a4 g8~ a b4~ g' f#2'.
e'4 e' e' d#' b1.
e'4 d' d' d' e' d'8~ c' b2.
d'4 e' d' d' g' f#' g'~ f#'8~ e' d#2'.
e'4 f#'8~ g' a'~ a b4 c'~ b8~ a g#1.
""", octave=2))

bass = @async play(parsevoice("""
e d c. b,8 a4,~ b, e2.
e4 a8~ g f#4 g8~ f# e~ f#~ g~ a~ b2~.
c4' b8~ a b4 b, e1.
e4 d c. b,8 a,4~ b, e2.
e4 a8~ g f#4 g8~ f# e~ f#~ g~ a~ b2~.
c4' b8~ a b4 b, e1.
e8~ f# g4 f# g c d g,2.
g4 f#8~ e f#4 g8~ f# e4 b, e4~ f# b,2.
e4 d c b, a,~ b, e1.
""", octave=2))   
end

Task (done) @0x00007f9fc4c67780

# A popular song from not too long ago

In [31]:
play(parsevoice("""
c''8 b' g'4 c''8 b' g'4 g'8 f' e' f'4 e'8 d' e'4 g' g'8~ g'2
c4 b,8 g, c c b, g, r
c''8 b' g'4  c''8 b' g'4 g'8 f' e' f'4 e'8 d' e'4 g' g'8~ g'2
c4 b,8 g, c c b, g, r
a4, a, e8 e f e r8 c c a, c4 c r
"""))

# Version data

In [21]:
versioninfo()

Julia Version 0.3.8-pre+5
Commit cfa638f (2015-04-15 05:30 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin14.3.0)
  CPU: Intel(R) Core(TM) i5-4258U CPU @ 2.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


In [22]:
Pkg.status()

479 required packages:
 - ASCIIPlots                    0.0.3
 - AWS                           0.1.8
 - AffineTransforms              0.0.5
 - AnsiColor                     0.0.2
 - AppleAccelerate               0.1.0
 - ApproxFun                     0.0.5
 - Arduino                       0.1.2
 - ArgParse                      0.2.10
 - ArrayViews                    0.4.10
 - Arrowhead                     0.0.1+             master
 - AudioIO                       0.1.0+             master
 - Autoreload                    0.2.0
 - AverageShiftedHistograms      0.1.0
 - BDF                           0.0.5
 - BEncode                       0.1.1
 - BSplines                      0.0.3
 - BackpropNeuralNet             0.0.3
 - BayesNets                     0.1.0
 - Benchmark                     0.1.0
 - BenchmarkLite                 0.1.2
 - BinDeps                       0.3.9
 - BioSeq                        0.3.0
 - Biryani                       0.2.0
 - Blink                         0.1.4